In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 라이브러리 설치 및 임포트
!pip install optuna -q

import pickle
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 17.1 MB/s eta 0:00:00


# 1. 데이터 로드

In [ ]:
# =============================================================================
# 데이터 로드
# =============================================================================
print("="*60)
print("📂 데이터 로드")
print("="*60)

LOAD_PATH = '/content/drive/MyDrive/preprocessed_data.pkl'

with open(LOAD_PATH, 'rb') as f:
    data_dict = pickle.load(f)

# Classification 데이터 추출
X_train = data_dict['X_train_clf']
X_val = data_dict['X_val_clf']
X_test = data_dict['X_test_clf']
y_train = data_dict['y_train_clf']
y_val = data_dict['y_val_clf']
y_test = data_dict['y_test_clf']

# Train + Val 합치기 (Optuna CV용)
X_train_full = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"✅ 데이터 로드 완료!")
print(f"   Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"   Train+Val: {X_train_full.shape}")

📂 데이터 로드
✅ 데이터 로드 완료!
   Train: (77366, 45), Val: (9660, 45), Test: (9670, 45)
   Train+Val: (87026, 45)


# 2. RandomForest: 기본

In [ ]:
print("\n" + "="*50)
print("🌲 Random Forest Classifier")
print("="*50)

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=1
)

rf_model.fit(X_train_full, y_train_full)
rf_pred = rf_model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, rf_pred):.4f}")
print(f"F1 Score (macro): {f1_score(y_test, rf_pred, average='macro'):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, rf_pred, target_names=['Bad', 'Standard', 'Good']))


🌲 Random Forest Classifier
Accuracy: 0.7764
F1 Score (macro): 0.7642

Classification Report:
              precision    recall  f1-score   support

         Bad       0.78      0.76      0.77      5497
    Standard       0.82      0.78      0.80     10301
        Good       0.66      0.79      0.72      3542

    accuracy                           0.78     19340
   macro avg       0.76      0.78      0.76     19340
weighted avg       0.78      0.78      0.78     19340



# 3. RandomForest: 최적화

## 3.1. 피처 엔지니어링

In [ ]:
# =============================================================================
# 1단계: 피처 엔지니어링
# =============================================================================
print("\n" + "="*60)
print("📌 1단계: 피처 엔지니어링")
print("="*60)

def create_features(df):
    data = df.copy()

    # 비율 피처
    data['debt_to_income'] = data['outstanding_debt'] / (data['annual_income'] + 1)
    data['emi_to_income'] = data['total_emi_per_month'] / (data['monthly_inhand_salary'] + 1)
    data['balance_to_income'] = data['monthly_balance'] / (data['monthly_inhand_salary'] + 1)
    data['invest_to_income'] = data['amount_invested_monthly'] / (data['monthly_inhand_salary'] + 1)

    # 신용 관련 복합 피처
    data['credit_load'] = data['num_credit_card'] * data['credit_utilization_ratio']
    data['delay_severity'] = data['delay_from_due_date'] * data['num_of_delayed_payment']
    data['loan_per_account'] = data['num_of_loan'] / (data['num_bank_accounts'] + 1)

    # 신용 이력 대비 피처
    data['inquiries_per_year'] = data['num_credit_inquiries'] / (data['credit_history_age'] / 12 + 1)
    data['accounts_per_year'] = data['num_bank_accounts'] / (data['credit_history_age'] / 12 + 1)

    # 지불 부담 점수
    data['payment_burden'] = data['total_emi_per_month'] + data['outstanding_debt'] * (data['interest_rate'] / 100)

    # 로그 변환
    data['log_income'] = np.log1p(data['annual_income'])
    data['log_debt'] = np.log1p(data['outstanding_debt'])

    return data

# 피처 엔지니어링 적용 (존재하는 컬럼만)
def safe_create_features(df):
    data = df.copy()
    cols = data.columns.tolist()

    if 'outstanding_debt' in cols and 'annual_income' in cols:
        data['debt_to_income'] = data['outstanding_debt'] / (data['annual_income'] + 1)
    if 'total_emi_per_month' in cols and 'monthly_inhand_salary' in cols:
        data['emi_to_income'] = data['total_emi_per_month'] / (data['monthly_inhand_salary'] + 1)
    if 'monthly_balance' in cols and 'monthly_inhand_salary' in cols:
        data['balance_to_income'] = data['monthly_balance'] / (data['monthly_inhand_salary'] + 1)
    if 'amount_invested_monthly' in cols and 'monthly_inhand_salary' in cols:
        data['invest_to_income'] = data['amount_invested_monthly'] / (data['monthly_inhand_salary'] + 1)
    if 'num_credit_card' in cols and 'credit_utilization_ratio' in cols:
        data['credit_load'] = data['num_credit_card'] * data['credit_utilization_ratio']
    if 'delay_from_due_date' in cols and 'num_of_delayed_payment' in cols:
        data['delay_severity'] = data['delay_from_due_date'] * data['num_of_delayed_payment']
    if 'num_of_loan' in cols and 'num_bank_accounts' in cols:
        data['loan_per_account'] = data['num_of_loan'] / (data['num_bank_accounts'] + 1)
    if 'num_credit_inquiries' in cols and 'credit_history_age' in cols:
        data['inquiries_per_year'] = data['num_credit_inquiries'] / (data['credit_history_age'] / 12 + 1)
    if 'num_bank_accounts' in cols and 'credit_history_age' in cols:
        data['accounts_per_year'] = data['num_bank_accounts'] / (data['credit_history_age'] / 12 + 1)
    if 'total_emi_per_month' in cols and 'outstanding_debt' in cols and 'interest_rate' in cols:
        data['payment_burden'] = data['total_emi_per_month'] + data['outstanding_debt'] * (data['interest_rate'] / 100)
    if 'annual_income' in cols:
        data['log_income'] = np.log1p(data['annual_income'])
    if 'outstanding_debt' in cols:
        data['log_debt'] = np.log1p(data['outstanding_debt'])

    return data

X_train_fe = safe_create_features(X_train_full)
X_test_fe = safe_create_features(X_test)

print(f"✅ 피처 엔지니어링 완료!")
print(f"   원본 피처: {X_train_full.shape[1]}개 → 새 피처 추가 후: {X_train_fe.shape[1]}개")

# =============================================================================
# 2단계-A: Feature Selection
# =============================================================================
print("\n" + "="*60)
print("📌 2단계-A: Feature Selection")
print("="*60)

rf_temp = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1)
rf_temp.fit(X_train_fe, y_train_full)

importance_df = pd.DataFrame({
    'feature': X_train_fe.columns,
    'importance': rf_temp.feature_importances_
}).sort_values('importance', ascending=False)

print("🔝 Top 15 피처:")
print(importance_df.head(15).to_string(index=False))

# 상위 20개 피처 선택
top_features = importance_df.head(20)['feature'].tolist()
X_train_sel = X_train_fe[top_features]
X_test_sel = X_test_fe[top_features]

print(f"\n✅ {len(top_features)}개 피처 선택 완료")


📌 1단계: 피처 엔지니어링
✅ 피처 엔지니어링 완료!
   원본 피처: 45개 → 새 피처 추가 후: 56개

📌 2단계-A: Feature Selection
🔝 Top 15 피처:
             feature  importance
          credit_mix    0.066740
            log_debt    0.053098
    outstanding_debt    0.052514
       interest_rate    0.050925
      payment_burden    0.049762
  inquiries_per_year    0.040678
   accounts_per_year    0.039129
changed_credit_limit    0.037344
         credit_load    0.036648
  credit_history_age    0.033031
      debt_to_income    0.028796
       emi_to_income    0.025738
     monthly_balance    0.025707
   balance_to_income    0.025703
     num_credit_card    0.025343

✅ 20개 피처 선택 완료


## 3.2. Optuna 하이퍼파라미터 튜닝

In [ ]:
# =============================================================================
# 2단계-B: Optuna 하이퍼파라미터 튜닝
# =============================================================================
print("\n" + "="*60)
print("📌 2단계-B: Optuna 하이퍼파라미터 튜닝")
print("="*60)

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200]),  # 범위 축소
        'max_depth': trial.suggest_categorical('max_depth', [10, 15, 20]),
        'min_samples_split': trial.suggest_categorical('min_samples_split', [2, 5]),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [1, 2]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None])
    }

    model = RandomForestClassifier(**params, random_state=42, n_jobs=1)
    scores = cross_val_score(model, X_train_sel, y_train_full, cv=2, scoring='f1_macro', n_jobs=1)  # cv=3→2
    return scores.mean()

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=10, show_progress_bar=True)  # 30→10
print(f"\n✅ 최적 파라미터: {study_rf.best_params}")
print(f"✅ 최고 CV F1 Score: {study_rf.best_value:.4f}")


📌 2단계-B: Optuna 하이퍼파라미터 튜닝


  0%|          | 0/10 [00:00<?, ?it/s]


✅ 최적 파라미터: {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'class_weight': None}
✅ 최고 CV F1 Score: 0.7534


## 3.3. Random Forest 학습 및 평가

In [ ]:
# =============================================================================
# 최종 Random Forest 학습 및 평가
# =============================================================================
print("\n" + "="*60)
print("🎯 최종 Random Forest 모델")
print("="*60)

best_rf = RandomForestClassifier(**study_rf.best_params, random_state=42, n_jobs=1)
best_rf.fit(X_train_sel, y_train_full)
rf_pred = best_rf.predict(X_test_sel)

print(f"\n📊 Test 성능:")
print(f"   Accuracy: {accuracy_score(y_test, rf_pred):.4f}")
print(f"   F1 Score (macro): {f1_score(y_test, rf_pred, average='macro'):.4f}")
print("\n" + classification_report(y_test, rf_pred, target_names=['Bad', 'Standard', 'Good']))

# =============================================================================
# 모델 저장
# =============================================================================
MODEL_SAVE_PATH = '/content/drive/MyDrive/best_rf_model.pkl'

save_dict = {
    'model': best_rf,
    'top_features': top_features,
    'best_params': study_rf.best_params
}

with open(MODEL_SAVE_PATH, 'wb') as f:
    pickle.dump(save_dict, f)

print(f"\n✅ 모델 저장 완료: {MODEL_SAVE_PATH}")


🎯 최종 Random Forest 모델

📊 Test 성능:
   Accuracy: 0.7967
   F1 Score (macro): 0.7850

              precision    recall  f1-score   support

         Bad       0.80      0.77      0.78      2749
    Standard       0.83      0.81      0.82      5150
        Good       0.71      0.80      0.75      1771

    accuracy                           0.80      9670
   macro avg       0.78      0.79      0.79      9670
weighted avg       0.80      0.80      0.80      9670


✅ 모델 저장 완료: /content/drive/MyDrive/best_rf_model.pkl
